In [8]:
import selenium
from selenium import webdriver
import urllib.request
import ssl

In [2]:
import time
# 相簿效期只有3個月，須判斷是否仍在效期內。（上傳日期：2018-07-29）

localtime = time.asctime( time.localtime(time.time()) )
# time.time() : seconds since 12:00am, January 1, 1970
# time.localtime() : show localtime in struct_time
# time.asctime() : show time in "Www Mmm dd hh:mm:ss yyyy" formate

dateStruct_time = time.strptime("2018-10-28 23:59:59", "%Y-%m-%d %H:%M:%S")
urlInvalidDate = time.asctime(dateStruct_time)

In [3]:
driver = webdriver.Chrome(r'D:\pyLadies\2018_0805_nmnsCrawler\chromedriver.exe')

urlDay1 = 'http://www.nmns.edu.tw/web/learn/feature/photo.htm?id=13'
urlDay2 = 'http://www.nmns.edu.tw/web/learn/feature/photo.htm?id=14'
urlDay3 = 'http://www.nmns.edu.tw/web/learn/feature/photo.htm?id=15'
day = 1
driver.get(urlDay1)

In [4]:
# 圖片網址可能所在之 xpath & attribute 對應
xpath_attribute = {
    '//div[@data-swiper-slide-index]':'style',
    '//div[@data-swiper-slide-index]/img':'src'
}

# urls be get
url_dict = {}

In [5]:
# method : get the clean URLs
def findURL(xpath,attribute):
    images = driver.find_elements_by_xpath(xpath)
    index = 0 #圖片張數計算

    for image in images:
        tagContent = image.get_attribute(attribute)
        
        if tagContent.find('\"') != -1: tagContent = tagContent.split('\"')[-1]
            # ↑split the string if it comes from attribute "style"

        if tagContent.find("https://") != -1:
            index += 1
            url = tagContent.split('\"')[0]
            # ↑split to get clear url from attribute "style"
            if url not in url_dict:
                url_dict[url] = "Day" + str(day) + "_" + str(index).zfill(3) + ".jpg"
            else:
                pass
        else:
            return False

In [10]:
# main
if localtime < urlInvalidDate:
    print("相簿已逾期！請手動確認檔案是否仍存在。")
else: # 找網址    
    for xpath,attribute in  xpath_attribute.items():
        if findURL(xpath,attribute) != False:
#             print("取得網址！")
            for key,value in url_dict.items():
                print(key,value)
                ssl._create_default_https_context = ssl._create_unverified_context
                    # ↑ to pass the SSL certificate verify
                resource = urllib.request.urlopen(key)
                output = open(value,"wb") # write binary
                output.write(resource.read())
                output.close()
        else:
            print( "注意：「", xpath, " + ", attribute, "」的組合失敗。" )

https://wwwadmin.nmns.edu.tw/upload/activity/1532917229RAyjbYoEoi.JPG Day1_031.jpg
https://wwwadmin.nmns.edu.tw/upload/activity/1532917228AkmF6LMCLo.JPG Day1_017.jpg
https://wwwadmin.nmns.edu.tw/upload/activity/1532917261l4tkjRHX7h.JPG Day1_051.jpg
https://wwwadmin.nmns.edu.tw/upload/activity/1532917210miYF0O15vk.jpg Day1_008.jpg
https://wwwadmin.nmns.edu.tw/upload/activity/1532917210zaPTMX2xfM.jpg Day1_007.jpg
https://wwwadmin.nmns.edu.tw/upload/activity/1532917261R4hPXInWxb.jpg Day1_053.jpg
https://wwwadmin.nmns.edu.tw/upload/activity/1532917277IAVNJNLl5z.JPG Day1_073.jpg
https://wwwadmin.nmns.edu.tw/upload/activity/1532917210tVPcc3oBYQ.JPG Day1_002.jpg
https://wwwadmin.nmns.edu.tw/upload/activity/1532917211ncKaBkdChv.JPG Day1_010.jpg
https://wwwadmin.nmns.edu.tw/upload/activity/15329172283WqW1YQzFB.JPG Day1_026.jpg
https://wwwadmin.nmns.edu.tw/upload/activity/1532917210pTYYPVA9Wz.jpg Day1_004.jpg
https://wwwadmin.nmns.edu.tw/upload/activity/1532917261goKuy5PTED.JPG Day1_060.jpg
http

In [11]:
driver.close()

url = "http://www.nmns.edu.tw/web/learn/feature/photo.htm?id=13" 

ctrl + /
